In [4]:
import cv2 as cv
import pandas as pd 
import numpy as np 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,AveragePooling2D,BatchNormalization,Dense,Dropout
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras import models
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import *

import random
random.seed(0)
print("Hello")

Hello


In [5]:
train_datagen=ImageDataGenerator(horizontal_flip=True,preprocessing_function=preprocess_input,
                                 width_shift_range=0.2,
                                 zoom_range=0.2,
                                height_shift_range=0.2,
                                 rotation_range=30)
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)


traindir="data/train"
testdir="data/val"

train_generator=train_datagen.flow_from_directory(
    traindir,
    target_size =(224,224),
    class_mode="categorical",
    batch_size=64
)

test_generator=test_datagen.flow_from_directory(
testdir,
target_size =(224,224),
class_mode="categorical",
batch_size=64
)

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)



print(train_generator.classes)
print(train_generator.class_indices)
print(class_weights)

Found 1533 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
[0 0 0 ... 1 1 1]
{'emergency': 0, 'noemergency': 1}
[1.23033708 0.84230769]


/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
resnet=DenseNet121(weights="imagenet",include_top=False,input_shape=(224,224,3))
model=models.Sequential()
model.add(resnet)
model.add(Flatten())
# model.add(Dropout(0.8))
model.add(Dense(2,activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 100354    
Total params: 7,137,858
Trainable params: 7,054,210
Non-trainable params: 83,648
_________________________________________________________________


In [16]:
# x,y = test_generator.next()
# for i in range(0,15):
#     image = x[i]
#     label = y[i]
#     print (label)
#     plt.imshow(image)
#     plt.show()

In [ ]:

sgd = optimizers.Nadam(lr=0.00001)
checkpoint = ModelCheckpoint('./models/model-{epoch:03d}.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')  
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = model.fit_generator( train_generator,
                              epochs=50, 
                              validation_data=test_generator,
                              class_weight={0:1.22503962,1:0.84480874},callbacks=[checkpoint])

In [ ]:
model.save("model.h5")